<a href="https://colab.research.google.com/github/kjawon/Bloomdent-AI/blob/main/%EB%AA%A8%EB%8D%B8_%ED%8C%8C%EC%9D%B4%ED%94%84%EB%9D%BC%EC%9D%B8_%EA%B2%80%EC%A6%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# OpenCV 라이브러리 설치
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 54.1 MB/s eta 0:00:00


In [ ]:
# ======================================================================
# [실전 모드] 원본 이미지 기반 AI 충치 진단기 (수정됨)
# ======================================================================
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
from torchvision import models, transforms
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from PIL import Image
from google.colab import drive
from tqdm.auto import tqdm
import glob
import random

# 구글 드라이브 마운트
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f">>> 작전 장비: {device}")

# ======================================================================
# 1. 경로 설정 (★수정 완료: Mydrive -> MyDrive★)
# ======================================================================
# 테스트할 이미지가 있는 폴더 (직접 만드시고 이미지를 넣으세요)
INPUT_DIR = '/content/UPdataset/validation/UPPERimages'
os.makedirs(INPUT_DIR, exist_ok=True)

# 결과가 저장될 폴더 (드라이브에 저장하고 싶으신 것 같아 수정했습니다)
# ★★★ 여기가 수정된 부분입니다 ★★★
OUTPUT_DIR = '/content/drive/MyDrive/모프/inference_results'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ★ 모델 경로 (Upper 모델 기준)
MASK_RCNN_PATH = '/content/drive/MyDrive/모프/학습모델/치아분할_MaskRCNN_upper_512_best.pth'
CARIES_RESNET_PATH = "/content/drive/MyDrive/모프/학습모델/caries_classifier_upper_224_balanced_best.pth"

# ... (이하 모델 정의 및 추론 코드는 동일합니다) ...

# ======================================================================
# 2. 모델 구조 정의
# ======================================================================
# --- Mask R-CNN ---
def get_model_instance_segmentation(num_classes):
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights=None)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)
    return model

# --- Caries Classifier (ResNet 224) ---
class CariesClassifier(nn.Module):
    def __init__(self, num_classes=1):
        super().__init__()
        try: self.base_model = models.resnet50(weights=None)
        except: self.base_model = models.resnet50(pretrained=False)

        self.base_model.fc = nn.Sequential(
            nn.Linear(self.base_model.fc.in_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, num_classes)
        )
    def forward(self, x): return self.base_model(x)

# ======================================================================
# 3. 모델 로드
# ======================================================================
def load_models():
    print(">>> 모델 로드 중...")

    # 1. Mask R-CNN
    det_model = get_model_instance_segmentation(num_classes=2).to(device)
    if os.path.exists(MASK_RCNN_PATH):
        det_model.load_state_dict(torch.load(MASK_RCNN_PATH, map_location=device))
        print("   - [완료] Mask R-CNN Loaded")
    else:
        print(f"   !!! 오류: Mask R-CNN 파일 없음 {MASK_RCNN_PATH}")
        return None, None
    det_model.eval()

    # 2. ResNet
    cls_model = CariesClassifier(1).to(device)
    if os.path.exists(CARIES_RESNET_PATH):
        try: cls_model.load_state_dict(torch.load(CARIES_RESNET_PATH, map_location=device))
        except: cls_model.load_state_dict(torch.load(CARIES_RESNET_PATH, map_location=device), strict=False)
        print("   - [완료] ResNet Loaded")
    else:
        print(f"   !!! 오류: ResNet 파일 없음 {CARIES_RESNET_PATH}")
        return None, None
    cls_model.eval()

    return det_model, cls_model

# ======================================================================
# 4. 실전 추론 파이프라인
# ======================================================================
def run_inference(detector, classifier, input_dir, output_dir):
    # 이미지 파일 찾기 (png, jpg, jpeg)
    image_files = glob.glob(os.path.join(input_dir, "*.png")) + \
                  glob.glob(os.path.join(input_dir, "*.jpg")) + \
                  glob.glob(os.path.join(input_dir, "*.jpeg"))

    if not image_files:
        print(f"\n!!! 경고: '{input_dir}' 폴더에 이미지가 없습니다.")
        print("    이미지를 폴더에 넣고 다시 실행해주세요.")
        return

    print(f"\n>>> 실전 분석 시작! (대상: {len(image_files)}개)")

    # 전처리
    t_det = transforms.Compose([transforms.ToTensor()]) # Mask R-CNN
    t_cls = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    for img_path in tqdm(image_files, desc="Processing"):
        basename = os.path.basename(img_path)

        try:
            # 이미지 로드
            img_bgr = cv2.imread(img_path)
            if img_bgr is None: continue
            img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
            vis_img = img_bgr.copy()
        except: continue

        # 1. [탐지] Mask R-CNN
        inp = t_det(Image.fromarray(img_rgb)).unsqueeze(0).to(device)
        with torch.no_grad():
            outputs = detector(inp)

        boxes = outputs[0]['boxes'].cpu().numpy()
        masks = outputs[0]['masks'].cpu().numpy()
        scores = outputs[0]['scores'].cpu().numpy()

        # 2. [분류 및 시각화]
        tooth_count = 0
        caries_count = 0

        for i, score in enumerate(scores):
            if score > 0.5: # 신뢰도 50% 이상인 박스만
                tooth_count += 1

                # 좌표 및 마스크
                x1, y1, x2, y2 = boxes[i].astype(int)
                mask_bin = (masks[i, 0] > 0.5).astype(np.uint8)

                # 크롭 (좌표 보정 포함)
                h, w = img_rgb.shape[:2]
                x1, y1 = max(0, x1), max(0, y1)
                x2, y2 = min(w, x2), min(h, y2)

                crop = img_rgb[y1:y2, x1:x2]
                if crop.size == 0: continue

                # 3. [진단] ResNet
                with torch.no_grad():
                    out = classifier(t_cls(Image.fromarray(crop)).unsqueeze(0).to(device))
                    prob = torch.sigmoid(out).item()

                is_caries = prob > 0.5

                # --- 시각화 ---
                if is_caries:
                    caries_count += 1
                    color = (0, 0, 255) # 빨강
                    status = "Caries"
                    conf_text = f"{prob*100:.1f}%"
                else:
                    color = (0, 255, 0) # 초록
                    status = "Normal"
                    conf_text = ""

                # 1) 마스크 색칠 (반투명)
                roi = vis_img[mask_bin == 1]
                if roi.size > 0:
                    vis_img[mask_bin == 1] = (roi * 0.6 + np.array(color) * 0.4).astype(np.uint8)

                # 2) 외곽선
                cnts, _ = cv2.findContours(mask_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                cv2.drawContours(vis_img, cnts, -1, color, 2)

                # 3) 박스 및 텍스트
                cv2.rectangle(vis_img, (x1, y1), (x2, y2), color, 2)
                label = f"{status} {conf_text}"

                (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
                cv2.rectangle(vis_img, (x1, y1 - 20), (x1 + tw, y1), color, -1)
                cv2.putText(vis_img, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

        # 결과 저장
        save_name = f"Result_{caries_count}Caries_{basename}"
        cv2.imwrite(os.path.join(output_dir, save_name), vis_img)

    print("\n" + "="*60)
    print(f"📢 [분석 완료]")
    print(f"   - 결과 저장소: {output_dir}")
    print(f"   - 분석한 이미지 수: {len(image_files)}장")
    print("="*60)

# ======================================================================
# 실행
# ======================================================================
det_net, cls_net = load_models()
if det_net and cls_net:
    run_inference(det_net, cls_net, INPUT_DIR, OUTPUT_DIR)

>>> 작전 장비: cuda
>>> 모델 로드 중...
   - [완료] Mask R-CNN Loaded
   - [완료] ResNet Loaded

>>> 실전 분석 시작! (대상: 3428개)


Processing:   0%|          | 0/3428 [00:00<?, ?it/s]


📢 [분석 완료]
   - 결과 저장소: /content/drive/MyDrive/모프/inference_results
   - 분석한 이미지 수: 3428장


In [ ]:
import os

# 1. 드라이브가 아직 마운트 안 되었다면 수행 (되어있다면 건너뜁니다)
if not os.path.exists('/content/drive'):
    from google.colab import drive
    drive.mount('/content/drive')

# --- 작전 경로 설정 ---

# 타겟 파일 경로 (구글 드라이브)
src_zip_path = '/content/drive/MyDrive/3.개방데이터/1.데이터/Validation/1.원천데이터/4.upper.zip'

# 압축을 풀 로컬 목적지 (Colab 내부)
# 이름이 겹치지 않게 'images'와 'labels'로 폴더를 나눴습니다.
src_dest = '/content/UPdataset/validation/UPPERimages'

# --- 작전 개시 (압축 해제) ---

print(f"작전 1: 원천 데이터(이미지) 해제 중... \n -> 목표지점: {src_dest}")
# 폴더 생성 (-p: 상위 폴더 없으면 같이 생성)
!mkdir -p "{src_dest}"
# 압축 해제 (-qq: 조용히, -d: 목적지 지정)
!unzip -qq "{src_zip_path}" -d "{src_dest}"


print("======== 작전 성공! 모든 데이터 전개 완료! ========")

작전 1: 원천 데이터(이미지) 해제 중... 
 -> 목표지점: /content/UPdataset/validation/UPPERimages
======== 작전 성공! 모든 데이터 전개 완료! ========


In [ ]:
# 충치 모델 검증 데이터를 저장할 대상 경로 설정
TARGET_DIR="/content/drive/MyDrive/모프/검증데이터/lower_nat"

# 1. 대상 경로 생성 (이미 존재하면 무시)
!mkdir -p "$TARGET_DIR"

# 2. ZIP 파일 압축 해제
# -d 옵션을 사용하여 압축 해제할 대상 디렉토리 지정
!unzip /content/drive/MyDrive/3.개방데이터/1.데이터/Validation/1.원천데이터/5.lower.zip -d "$TARGET_DIR"

# 3. 압축 해제 결과 확인 (선택 사항)
!ls "$TARGET_DIR"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/drive/MyDrive/모프/검증데이터/lower_nat/lower_39563.png  
  inflating: /content/drive/MyDrive/모프/검증데이터/lower_nat/lower_39564.png  
  inflating: /content/drive/MyDrive/모프/검증데이터/lower_nat/lower_39565.png  
  inflating: /content/drive/MyDrive/모프/검증데이터/lower_nat/lower_39566.png  
  inflating: /content/drive/MyDrive/모프/검증데이터/lower_nat/lower_39567.png  
  inflating: /content/drive/MyDrive/모프/검증데이터/lower_nat/lower_39568.png  
  inflating: /content/drive/MyDrive/모프/검증데이터/lower_nat/lower_39569.png  
  inflating: /content/drive/MyDrive/모프/검증데이터/lower_nat/lower_39570.png  
  inflating: /content/drive/MyDrive/모프/검증데이터/lower_nat/lower_39571.png  
  inflating: /content/drive/MyDrive/모프/검증데이터/lower_nat/lower_39572.png  
  inflating: /content/drive/MyDrive/모프/검증데이터/lower_nat/lower_39573.png  
  inflating: /content/drive/MyDrive/모프/검증데이터/lower_nat/lower_39574.png  
  inflating: /content/drive/MyDrive/모프/검증데이터/lower_nat/lower_39575.png  
  inflating: /c

In [ ]:
# ======================================================================
# 1. 환경 설정 및 라이브러리
# ======================================================================
import os
import cv2
import torch
import torch.nn as nn
import numpy as np
from torchvision import transforms
from PIL import Image
from google.colab import drive
from tqdm.auto import tqdm
import glob
import random

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f">>> 현재 작전 장비: {device}")

# ======================================================================
# 2. 경로 설정 (FRONT 데이터)
# ======================================================================
# 원본 이미지 경로
IMAGE_DIR = '/content/drive/MyDrive/3.개방데이터/1.데이터/Training/1.원천데이터/2.front'
# 모델 경로
TOOTH_MODEL_PATH = "/content/drive/MyDrive/모프/학습모델/치아좌표탐지모델_unet_pytorch_best.pth"
# 결과 저장 경로 (디버깅용 폴더)
DEBUG_SAVE_DIR = "/content/drive/MyDrive/모프/검증결과/디버깅_마스크_확인"

# ======================================================================
# 3. 모델 구조 (U-Net) - 오류 방지를 위해 포함
# ======================================================================
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True)
        )
    def forward(self, x): return self.double_conv(x)

class Down(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(nn.MaxPool2d(2), DoubleConv(in_channels, out_channels))
    def forward(self, x): return self.maxpool_conv(x)

class Up(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
        self.conv = DoubleConv(in_channels, out_channels)
    def forward(self, x1, x2):
        x1 = self.up(x1)
        dy = x2.size()[2]-x1.size()[2]; dx = x2.size()[3]-x1.size()[3]
        x1 = torch.nn.functional.pad(x1, [dx//2, dx-dx//2, dy//2, dy-dy//2])
        return self.conv(torch.cat([x2, x1], dim=1))

class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
    def forward(self, x): return self.conv(x)

class UNet(nn.Module):
    def __init__(self, n_channels=3, n_classes=1):
        super().__init__()
        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128); self.down2 = Down(128, 256)
        self.down3 = Down(256, 512); self.down4 = Down(512, 1024)
        self.up1 = Up(1024, 512); self.up2 = Up(512, 256)
        self.up3 = Up(256, 128); self.up4 = Up(128, 64)
        self.outc = OutConv(64, n_classes)
    def forward(self, x):
        x1=self.inc(x); x2=self.down1(x1); x3=self.down2(x2); x4=self.down3(x3); x5=self.down4(x4)
        x=self.up1(x5,x4); x=self.up2(x,x3); x=self.up3(x,x2); x=self.up4(x,x1)
        return self.outc(x)

# ======================================================================
# 4. 마스크 추출 및 저장 함수
# ======================================================================
def extract_and_save_masks(num_samples=50):
    print(f">>> 🕵️‍♂️ 마스크 상태 정밀 진단 시작 (샘플링: {num_samples}장)")

    # 저장 폴더 생성
    os.makedirs(DEBUG_SAVE_DIR, exist_ok=True)

    # 모델 로드
    model = UNet(3, 1).to(device)
    if os.path.exists(TOOTH_MODEL_PATH):
        model.load_state_dict(torch.load(TOOTH_MODEL_PATH, map_location=device))
        print("   - 모델 로드 완료")
    else:
        print("!!! 모델 파일이 없습니다.")
        return
    model.eval()

    # 이미지 파일 리스트 가져오기 (랜덤 샘플링)
    all_images = glob.glob(os.path.join(IMAGE_DIR, "*.png"))
    if len(all_images) > num_samples:
        target_files = random.sample(all_images, num_samples)
    else:
        target_files = all_images

    transform = transforms.Compose([transforms.Resize((128, 128)), transforms.ToTensor()])

    for img_path in tqdm(target_files, desc="Generating Masks"):
        try:
            # 이미지 로드
            img_bgr = cv2.imread(img_path)
            if img_bgr is None: continue
            img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
            h, w = img_bgr.shape[:2]
            basename = os.path.splitext(os.path.basename(img_path))[0]

            # 추론 (Inference)
            inp = transform(Image.fromarray(img_rgb)).unsqueeze(0).to(device)
            with torch.no_grad():
                # Sigmoid를 통과시켜 확률맵(0~1) 생성
                mask_prob = torch.sigmoid(model(inp)).cpu().numpy()[0, 0]

            # 원본 크기로 복원 (중요: 여기서 떡짐 현상이 가속화될 수 있음)
            mask_resized = cv2.resize(mask_prob, (w, h), interpolation=cv2.INTER_LINEAR)

            # 이진화 (Threshold 0.5) -> 0 또는 255
            mask_binary = (mask_resized > 0.5).astype(np.uint8) * 255

            # 1. 순수 마스크 저장 (흑백)
            cv2.imwrite(os.path.join(DEBUG_SAVE_DIR, f"Pure_Mask_{basename}.png"), mask_binary)

            # 2. 오버레이 이미지 생성 (빨간색 마스크)
            # 마스크가 있는 부분만 빨간색을 입힘
            overlay = img_bgr.copy()
            # BGR에서 Red 채널(2)만 255로, 나머지는 유지하면서 투명도 적용
            overlay[mask_binary == 255] = [0, 0, 255]

            # 원본과 오버레이 합성 (투명도 0.4)
            final_vis = cv2.addWeighted(overlay, 0.4, img_bgr, 0.6, 0)

            cv2.imwrite(os.path.join(DEBUG_SAVE_DIR, f"Overlay_{basename}.jpg"), final_vis)

        except Exception as e:
            print(f"Error processing {img_path}: {e}")
            continue

    print("\n" + "="*60)
    print("📢 [진단 완료]")
    print(f"   - 저장 경로: {DEBUG_SAVE_DIR}")
    print("   - 확인 방법: 'Pure_Mask' 이미지를 열어 치아가 분리되어 있는지,")
    print("                하나의 덩어리(Blob)로 뭉쳐 있는지 확인하십시오.")
    print("="*60)

# 실행
extract_and_save_masks(num_samples=20) # 20장만 빠르게 확인

>>> 현재 작전 장비: cuda
>>> 🕵️‍♂️ 마스크 상태 정밀 진단 시작 (샘플링: 20장)
   - 모델 로드 완료


Generating Masks:   0%|          | 0/20 [00:00<?, ?it/s]


📢 [진단 완료]
   - 저장 경로: /content/drive/MyDrive/모프/검증결과/디버깅_마스크_확인
   - 확인 방법: 'Pure_Mask' 이미지를 열어 치아가 분리되어 있는지,
                하나의 덩어리(Blob)로 뭉쳐 있는지 확인하십시오.


In [ ]:
import os
import json
import glob
from tqdm import tqdm

# 검증용 JSON 파일들이 위치한 경로
JSON_DIR = '/content/drive/MyDrive/모프/검증데이터/front_val'

def inspect_caries_data(json_dir):
    json_files = glob.glob(os.path.join(json_dir, "*.json"))
    print(f">>> 정밀 수색 시작: 총 {len(json_files)}개의 파일 분석 중...\n")

    total_teeth = 0
    decayed_count = 0
    normal_count = 0

    # 충치가 포함된 파일명을 저장할 리스트
    files_with_caries = []

    for json_path in tqdm(json_files, desc="분석 중"):
        try:
            with open(json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            has_decayed_in_file = False

            # 각 파일 내의 치아 정보 순회
            for tooth in data.get('tooth', []):
                total_teeth += 1
                is_decayed = tooth.get('decayed', False)

                if is_decayed:
                    decayed_count += 1
                    has_decayed_in_file = True
                else:
                    normal_count += 1

            # 이 파일에 충치 치아가 하나라도 있었다면 목록에 추가
            if has_decayed_in_file:
                files_with_caries.append(os.path.basename(json_path))

        except Exception as e:
            print(f"!!! 파일 읽기 오류: {json_path} - {e}")

    # === 최종 보고 ===
    print("\n" + "="*50)
    print("📊 [데이터셋 분석 결과 보고]")
    print(f"   - 총 파일 수: {len(json_files)}개")
    print(f"   - 총 치아 수: {total_teeth}개")
    print("-" * 30)
    print(f"   ✅ 정상 치아 (Normal): {normal_count}개")
    print(f"   🦠 충치 치아 (Caries): {decayed_count}개")
    print("-" * 30)

    if total_teeth > 0:
        decay_ratio = (decayed_count / total_teeth) * 100
        print(f"   📈 충치 비율: {decay_ratio:.2f}%")

    if decayed_count == 0:
        print("\n🚨 [긴급] 이 데이터셋에는 충치(True) 라벨이 하나도 없습니다!")
        print("   -> 검증 데이터셋을 교체하거나 라벨링을 확인해야 합니다.")
    else:
        print(f"\n📂 충치가 발견된 파일 목록 ({len(files_with_caries)}개):")
        # 너무 많으면 앞의 10개만 출력
        for i, fname in enumerate(files_with_caries[:10]):
            print(f"   {i+1}. {fname}")
        if len(files_with_caries) > 10:
            print(f"   ... 외 {len(files_with_caries)-10}개 파일")

    print("="*50)

# 함수 실행
inspect_caries_data(JSON_DIR)

>>> 정밀 수색 시작: 총 1994개의 파일 분석 중...



분석 중: 100%|██████████| 1994/1994 [00:21<00:00, 92.42it/s] 


📊 [데이터셋 분석 결과 보고]
   - 총 파일 수: 1994개
   - 총 치아 수: 31500개
------------------------------
   ✅ 정상 치아 (Normal): 31500개
   🦠 충치 치아 (Caries): 0개
------------------------------
   📈 충치 비율: 0.00%

🚨 [긴급] 이 데이터셋에는 충치(True) 라벨이 하나도 없습니다!
   -> 검증 데이터셋을 교체하거나 라벨링을 확인해야 합니다.


In [ ]:
import os
import json
import glob
from tqdm import tqdm

# 경로 설정 (Front 데이터)
FULL_JSON_DIR = '/content/drive/MyDrive/모프 구강 데이터/front'
TRAIN_LIST_PATH = '/content/drive/MyDrive/모프/파일리스트_front/file_pairs_list_50k.json'

def find_unseen_caries_files(json_dir, train_list_path):
    print(">>> 🕵️‍♂️ '미사용 충치 데이터' 정밀 수색 시작...")

    # 1. 학습 파일명 집합 생성
    train_filenames = set()
    if os.path.exists(train_list_path):
        with open(train_list_path, 'r') as f:
            train_data = json.load(f)
        for item in train_data:
            fname = os.path.splitext(os.path.basename(item['image_path']))[0]
            train_filenames.add(fname)
    else:
        print("!!! 학습 리스트 파일이 없습니다.")
        return

    # 2. 전체 파일 스캔
    all_files = glob.glob(os.path.join(json_dir, "*.json"))
    unseen_caries_files = []
    unseen_normal_count = 0

    for json_path in tqdm(all_files, desc="Searching"):
        base_name = os.path.splitext(os.path.basename(json_path))[0]

        # 학습에 안 쓴 파일(Unseen)만 확인
        if base_name not in train_filenames:
            try:
                with open(json_path, 'r') as f:
                    data = json.load(f)

                # 충치가 하나라도 있는지 확인
                has_caries = False
                for tooth in data.get('tooth', []):
                    if tooth.get('decayed', False):
                        has_caries = True
                        break

                if has_caries:
                    unseen_caries_files.append(json_path)
                else:
                    unseen_normal_count += 1

            except: continue

    # 3. 결과 보고
    print("\n" + "="*50)
    print(f"🔍 [수색 결과 보고]")
    print(f"   - 학습에 안 쓴 파일 총 개수: {unseen_normal_count + len(unseen_caries_files)}개")
    print("-" * 30)
    print(f"   ✅ 정상 파일 (충치 X): {unseen_normal_count}개")
    print(f"   🦠 충치 포함 파일 (충치 O): {len(unseen_caries_files)}개")
    print("="*50)

    if len(unseen_caries_files) == 0:
        print("\n🚨 [비상] 학습 안 한 데이터에는 충치가 하나도 없습니다!")
        print("   -> 해결책: 학습 데이터(50k) 중에서 일부(Test set)를 떼어내서 검증해야 합니다.")
    else:
        print(f"\n>>> 발견된 미사용 충치 파일 {len(unseen_caries_files)}개를 사용하여 다시 검증하시겠습니까?")
        return unseen_caries_files

# 실행
unseen_caries_list = find_unseen_caries_files(FULL_JSON_DIR, TRAIN_LIST_PATH)

>>> 🕵️‍♂️ '미사용 충치 데이터' 정밀 수색 시작...


Searching: 100%|██████████| 15952/15952 [05:23<00:00, 49.26it/s] 


🔍 [수색 결과 보고]
   - 학습에 안 쓴 파일 총 개수: 12799개
------------------------------
   ✅ 정상 파일 (충치 X): 10811개
   🦠 충치 포함 파일 (충치 O): 1988개

>>> 발견된 미사용 충치 파일 1988개를 사용하여 다시 검증하시겠습니까?


In [ ]:
# ======================================================================
# [1단계] 치아 탐지 및 크롭 저장 (512x512 모델 사용)
# ======================================================================
import os
import json
import cv2
import numpy as np
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
from tqdm.auto import tqdm
import glob
import random

# 구글 드라이브 마운트
if not os.path.exists('/content/drive'):
    from google.colab import drive
    drive.mount('/content/drive')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f">>> 현재 작전 장비: {device}")

# ----------------------------------------------------------------------
# 1. 설정 (경로 확인)
# ----------------------------------------------------------------------
IMAGE_DIR = '/content/drive/MyDrive/3.개방데이터/1.데이터/Training/1.원천데이터/2.front'
JSON_DIR = '/content/drive/MyDrive/모프 구강 데이터/front'
TRAIN_LIST_PATH = '/content/drive/MyDrive/모프/파일리스트_front/file_pairs_list_50k.json'

# ★ 탐지 모델: 512 해상도 버전 사용
TOOTH_MODEL_PATH = '/content/drive/MyDrive/모프/학습모델/치아좌표탐지모델_unet_front_512.pth'

# ★ 중간 데이터 저장소 (다리 역할)
INTERMEDIATE_DIR = "/content/drive/MyDrive/모프/중간_크롭_데이터_128용"
os.makedirs(INTERMEDIATE_DIR, exist_ok=True)

# ----------------------------------------------------------------------
# 2. U-Net 모델 구조 (학습 코드와 동일)
# ----------------------------------------------------------------------
class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.double_conv = nn.Sequential(nn.Conv2d(in_ch, out_ch, 3, 1, 1), nn.BatchNorm2d(out_ch), nn.ReLU(inplace=True), nn.Conv2d(out_ch, out_ch, 3, 1, 1), nn.BatchNorm2d(out_ch), nn.ReLU(inplace=True))
    def forward(self, x): return self.double_conv(x)
class Down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__(); self.maxpool_conv = nn.Sequential(nn.MaxPool2d(2), DoubleConv(in_ch, out_ch))
    def forward(self, x): return self.maxpool_conv(x)
class Up(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__(); self.up = nn.ConvTranspose2d(in_ch, in_ch//2, 2, 2); self.conv = DoubleConv(in_ch, out_ch)
    def forward(self, x1, x2):
        x1 = self.up(x1); diffY = x2.size()[2]-x1.size()[2]; diffX = x2.size()[3]-x1.size()[3]
        x1 = torch.nn.functional.pad(x1, [diffX//2, diffX-diffX//2, diffY//2, diffY-diffY//2]); return self.conv(torch.cat([x2, x1], dim=1))
class OutConv(nn.Module):
    def __init__(self, in_ch, out_ch): super().__init__(); self.conv = nn.Conv2d(in_ch, out_ch, 1)
    def forward(self, x): return self.conv(x)
class UNet(nn.Module):
    def __init__(self, n_channels=3, n_classes=1):
        super().__init__(); self.inc = DoubleConv(n_channels, 64); self.down1 = Down(64, 128); self.down2 = Down(128, 256); self.down3 = Down(256, 512); self.down4 = Down(512, 1024); self.up1 = Up(1024, 512); self.up2 = Up(512, 256); self.up3 = Up(256, 128); self.up4 = Up(128, 64); self.outc = OutConv(64, n_classes)
    def forward(self, x): x1=self.inc(x); x2=self.down1(x1); x3=self.down2(x2); x4=self.down3(x3); x5=self.down4(x4); x=self.up1(x5,x4); x=self.up2(x,x3); x=self.up3(x,x2); x=self.up4(x,x1); return self.outc(x)

# ----------------------------------------------------------------------
# 3. 워터쉐드 및 유틸리티
# ----------------------------------------------------------------------
def separate_teeth_watershed(mask_binary):
    dist_transform = cv2.distanceTransform(mask_binary, cv2.DIST_L2, 5)
    ret, sure_fg = cv2.threshold(dist_transform, 0.3 * dist_transform.max(), 255, 0) # 임계값 0.3
    sure_fg = np.uint8(sure_fg)
    sure_bg = cv2.dilate(mask_binary, np.ones((3,3), np.uint8), iterations=3)
    unknown = cv2.subtract(sure_bg, sure_fg)
    ret, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1
    markers[unknown == 255] = 0
    dummy_img = cv2.cvtColor(mask_binary, cv2.COLOR_GRAY2BGR)
    markers = cv2.watershed(dummy_img, markers)

    boxes = []
    for label in np.unique(markers):
        if label <= 1: continue
        temp_mask = np.zeros_like(mask_binary)
        temp_mask[markers == label] = 255
        cnts, _ = cv2.findContours(temp_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for c in cnts:
            x, y, w, h = cv2.boundingRect(c)
            if w > 15 and h > 15: boxes.append([x, y, w, h])
    return boxes

def calculate_iou(boxA, boxB):
    xA, yA = max(boxA[0], boxB[0]), max(boxA[1], boxB[1])
    xB, yB = min(boxA[0]+boxA[2], boxB[0]+boxB[2]), min(boxA[1]+boxA[3], boxB[1]+boxB[3])
    inter = max(0, xB-xA) * max(0, yB-yA)
    return inter / float(boxA[2]*boxA[3] + boxB[2]*boxB[3] - inter + 1e-6)

# ----------------------------------------------------------------------
# 4. 1단계 실행 함수
# ----------------------------------------------------------------------
def run_step1():
    print(">>> [1단계] 치아 탐지 및 크롭 시작...")

    # 모델 로드
    model = UNet(3, 1).to(device)
    if os.path.exists(TOOTH_MODEL_PATH):
        model.load_state_dict(torch.load(TOOTH_MODEL_PATH, map_location=device))
        print(f"   - 512 탐지 모델 로드 완료")
    else:
        print("!!! 탐지 모델 파일이 없습니다.")
        return
    model.eval()

    # 데이터 선별 (Unseen)
    train_set = set()
    if os.path.exists(TRAIN_LIST_PATH):
        with open(TRAIN_LIST_PATH) as f:
            for i in json.load(f): train_set.add(os.path.splitext(os.path.basename(i['image_path']))[0])

    all_files = glob.glob(os.path.join(JSON_DIR, "*.json"))
    target_files = []
    for p in all_files:
        if os.path.splitext(os.path.basename(p))[0] not in train_set:
            target_files.append(p)

    # 테스트용으로 1000개만 제한 (원하시면 이 줄을 주석 처리하세요)
    target_files = target_files[:1000]
    print(f"   - 처리 대상 파일: {len(target_files)}개")

    transform = transforms.Compose([transforms.Resize((512, 512)), transforms.ToTensor()])
    count = 0

    for json_path in tqdm(target_files, desc="Step 1 Processing"):
        base = os.path.splitext(os.path.basename(json_path))[0]
        img_path = os.path.join(IMAGE_DIR, base + ".png")
        if not os.path.exists(img_path): continue

        try:
            img_bgr = cv2.imread(img_path)
            if img_bgr is None: continue
            img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
            h, w = img_bgr.shape[:2]
        except: continue

        # 추론 (512)
        inp = transform(Image.fromarray(img_rgb)).unsqueeze(0).to(device)
        with torch.no_grad():
            mask = (torch.sigmoid(model(inp)) > 0.5).float().cpu().numpy()[0, 0]

        # 워터쉐드 분할
        mask_uint8 = cv2.resize(mask, (w, h), interpolation=cv2.INTER_NEAREST).astype(np.uint8) * 255
        pred_boxes = separate_teeth_watershed(mask_uint8)

        # GT 매칭 및 크롭
        with open(json_path) as f: data = json.load(f)

        for t in data.get('tooth', []):
            gseg = np.array(t.get('segmentation', []), np.int32)
            if len(gseg)==0: continue
            gbox = cv2.boundingRect(gseg)
            gdec = 1 if t.get('decayed') else 0
            t_num = t.get('teeth_num', 0)

            best_iou, best_box = 0, None
            for pb in pred_boxes:
                iou = calculate_iou(gbox, pb)
                if iou > best_iou: best_iou, best_box = iou, pb

            if best_iou > 0.1 and best_box:
                bx, by, bw, bh = best_box
                crop = img_rgb[by:by+bh, bx:bx+bw]
                if crop.size == 0: continue

                # 저장: 파일명에 정답(GT) 포함
                save_name = f"{base}_{t_num}_GT{gdec}.png"
                cv2.imwrite(os.path.join(INTERMEDIATE_DIR, save_name), cv2.cvtColor(crop, cv2.COLOR_RGB2BGR))
                count += 1

    print(f"\n>>> [1단계 완료] {count}개의 치아 이미지가 '{INTERMEDIATE_DIR}'에 저장되었습니다.")

# 1단계 실행
run_step1()

>>> 현재 작전 장비: cuda
>>> [1단계] 치아 탐지 및 크롭 시작...
   - 512 탐지 모델 로드 완료
   - 처리 대상 파일: 1000개


Step 1 Processing:   0%|          | 0/1000 [00:00<?, ?it/s]


>>> [1단계 완료] 14018개의 치아 이미지가 '/content/drive/MyDrive/모프/중간_크롭_데이터_128용'에 저장되었습니다.


In [ ]:
# ======================================================================
# [2단계] 충치 분류 및 성능 평가 (128x128 모델 사용)
# ======================================================================
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# ----------------------------------------------------------------------
# 1. 설정
# ----------------------------------------------------------------------
# ★ 분류 모델: 기존 128 해상도 버전 사용
CARIES_MODEL_PATH = "/content/drive/MyDrive/모프/학습모델/caries_detection_model_front_50k_best_pytorch.pth"
# 중간 데이터 저장소 (1단계에서 생성된 곳)
INTERMEDIATE_DIR = "/content/drive/MyDrive/모프/중간_크롭_데이터_128용"

# ----------------------------------------------------------------------
# 2. ResNet 모델 구조 (128 학습 때 사용한 구조)
# ----------------------------------------------------------------------
class CariesClassifier(nn.Module):
    def __init__(self, num_classes=1):
        super().__init__()
        try: self.base_model = models.resnet50(weights='DEFAULT')
        except: self.base_model = models.resnet50(weights='IMAGENET1K_V1')

        # ★ 중요: 기존 128 모델의 FC 레이어 구조와 일치시켜야 함
        # (가중치 로드 에러 방지)
        num_ftrs = self.base_model.fc.in_features
        self.base_model.fc = nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes),
            nn.Sigmoid() # 기존 모델은 Sigmoid가 포함되어 있을 확률이 높음
        )
    def forward(self, x): return self.base_model(x)

# ----------------------------------------------------------------------
# 3. 2단계 실행 함수
# ----------------------------------------------------------------------
def run_step2():
    print("\n>>> [2단계] 충치 분류 및 채점 시작...")

    # 모델 로드
    model = CariesClassifier(1).to(device)
    if os.path.exists(CARIES_MODEL_PATH):
        # Strict=False를 사용하여 미세한 레이어 이름 차이가 있어도 로드 시도
        # (만약 fc 레이어 이름이 안 맞으면 오류 날 수 있으니 확인 필요)
        try:
            model.load_state_dict(torch.load(CARIES_MODEL_PATH, map_location=device))
        except:
            # 키 불일치 시 'model' 키 확인
            state = torch.load(CARIES_MODEL_PATH, map_location=device)
            if 'model' in state: model.load_state_dict(state['model'])
            else: model.load_state_dict(state, strict=False)

        print(f"   - 128 분류 모델 로드 완료")
    else:
        print("!!! 분류 모델 파일이 없습니다.")
        return
    model.eval()

    # 데이터 로드
    crop_files = glob.glob(os.path.join(INTERMEDIATE_DIR, "*.png"))
    if not crop_files:
        print("!!! 검증할 데이터가 없습니다. 1단계를 먼저 실행하세요.")
        return
    print(f"   - 검증 대상: {len(crop_files)}개 치아")

    # 전처리 (128 해상도)
    transform = transforms.Compose([
        transforms.Resize((128, 128)), # ★ 128로 리사이즈
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    y_true = []
    y_pred = []

    for path in tqdm(crop_files, desc="Step 2 Classifying"):
        try:
            # 파일명에서 정답(GT) 추출: image_11_GT1.png -> 1
            filename = os.path.basename(path)
            gt_part = filename.split("_GT")[1] # "1.png"
            gt_label = int(gt_part.split(".")[0]) # 1

            # 추론
            img = Image.open(path).convert("RGB")
            inp = transform(img).unsqueeze(0).to(device)

            with torch.no_grad():
                out = model(inp) # Sigmoid가 포함되어 있다면 0~1 값 출력
                prob = out.item()
                pred = 1 if prob > 0.5 else 0

            y_true.append(gt_label)
            y_pred.append(pred)
        except Exception as e:
            continue

    # 결과 출력
    print("\n" + "="*60)
    if len(y_true) > 0:
        print("[최종 성적표 (128 ResNet)]")
        print(classification_report(y_true, y_pred, target_names=['Normal', 'Caries']))

        cm = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(6, 5))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Normal', 'Caries'], yticklabels=['Normal', 'Caries'])
        plt.title('Confusion Matrix (128 ResNet)')
        plt.show()
    else:
        print("!!! 유효한 결과가 없습니다.")
    print("="*60)

# 2단계 실행
run_step2()


>>> [2단계] 충치 분류 및 채점 시작...
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 221MB/s]


   - 128 분류 모델 로드 완료
   - 검증 대상: 14018개 치아


Step 2 Classifying:   0%|          | 0/14018 [00:00<?, ?it/s]

KeyboardInterrupt: 